In [1]:
#this script will create a graph to be used for the dfs 
#for future, we can develop JSON file saving method instead of using pickle

In [2]:
import pickle
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time
from datetime import datetime, timedelta

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)



In [3]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
trip_table_fixed =  pd.read_csv(r'trip_table_fixed.csv', sep=",", engine='python')

In [4]:
########OPTIONAL##########
#select only the 2 hour time frame of 7-9 am from trip_table_fixed
#if the tables are not shrunk, the dfs module will take exponentially longer, and may not finish
trip_table_fixed = trip_table_fixed[trip_table_fixed['arrival_time_fix'].between(25200, 32400, inclusive=True)]



In [5]:
#create graph
graph = {}
unique_table=trip_table_fixed.groupby(['full_bus_name','trip_id', 'stop_id_time']).size().reset_index()
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    graph[bus_name] = {}
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    bus_trip_id = unique_table.loc[index, 'trip_id']
    stop_id = unique_table.loc[index, 'stop_id_time']    
    graph[bus_name][bus_trip_id] = set([])
graph_ready = copy.deepcopy(graph)

In [6]:
#create graph form 
#bus#:set([bus_stops])
for index, row in trip_table_fixed.iterrows():
    route_name = row['full_bus_name']
    stop_id = row['stop_id_time']
    trip_id = row['trip_id']
    try:
        graph_ready[route_name][trip_id].add(stop_id)
    except:
        print(route_name, stop_id, trip_id)
        break

In [7]:
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
for key in graph_ready:
    for trip_id in graph_ready[key]:                   
        graph_temp = sorted(list(graph_ready[key][trip_id]), key=lambda x: int(x.split('_')[1]))
        graph_ready[key][trip_id] = graph_temp



In [1]:
# graph_ready

In [8]:
#graph_ready
save_obj(graph_ready, 'graph_ready')

In [10]:
for id in graph_ready:
    if id == 904846:
        print(graph_ready[id])

{'10_DROP OFFS ONLY': {886625: ['941_32220'],
  886626: ['313_31500'],
  886627: ['1301_31620'],
  886628: ['313_30540'],
  886629: ['313_29040'],
  886630: ['313_27960'],
  886631: ['941_27720'],
  886636: ['313_26940'],
  886687: ['2576_27796'],
  886688: ['2576_28345'],
  886689: ['2376_29160'],
  886690: ['2576_29476'],
  886691: ['2576_29785'],
  886692: ['2376_30840'],
  886693: ['2576_31585'],
  886694: ['2576_31045'],
  886720: ['2398_31440'],
  886721: ['2576_31156'],
  886722: ['2576_31885'],
  886724: ['2576_31696'],
  896783: ['313_26940'],
  896788: ['941_27720'],
  896789: ['313_27960'],
  896790: ['313_29040'],
  896791: ['313_30540'],
  896792: ['1301_31620'],
  896793: ['313_31500'],
  896794: ['941_32220'],
  896829: ['2770_31620', '2576_31696'],
  896830: ['2770_32340'],
  896831: ['2770_31800', '2576_31885'],
  896832: ['2770_31080', '2576_31156'],
  896833: ['2398_31440'],
  896859: ['2770_30960', '2576_31045'],
  896860: ['2770_31500', '2576_31585'],
  896861: ['2

In [17]:
count = 0 
count_list = []
graph_final = {}
node = trip_table_fixed['stop_id_time'].unique()
for stop_id_time in node:
    stop_id = stop_id_time.split('_')[0]
    stop_time = int(stop_id_time.split('_')[1]) #current time
    #wait time is 30 min 60*30
    stop_time_30 = stop_time+1800
    graph_final[stop_id_time] = [] 
    for bus_name in graph_ready:
        for trip_id in graph_ready[bus_name]:
            stops_list = graph_ready[bus_name][trip_id]
            stops_only_list =[stops_with_time.split('_', 1)[0] for stops_with_time in stops_list]

            #gets the first hit. But a trip_id should not have multiple same stops
            try:
                index_of_stop = stops_only_list.index(stop_id)
            except:
                continue
            matching = [s for s in stops_only_list if stop_id == s]

            #check if the trip_id could possibly have two hits on a stop_id
            if len(matching) > 1: 
                print('error - trip_id has multiple same stop_ids')
                print(stop_id,bus_name,trip_id,stops_list)
                pass

            current_stop = stops_list[index_of_stop]
            current_stop_time = int(current_stop.split('_')[1])
            #check if the stop time for that bus is within 30 min wait
            if stop_time <= current_stop_time and current_stop_time <= stop_time_30: 
                #if this trip_id is the last stop, continue to the next trip_id
                try:
                    next_stop = stops_list[index_of_stop+1]
                    next_stop_id = next_stop.split('_')[0]
                    next_stop_time = next_stop.split('_')[1]
                except: 
                    continue
                travel_time = int(next_stop_time) - int(stop_time)    
                dic = {'destination': next_stop, 'travel_time': travel_time, 'trip_id': trip_id, 'bus_name': bus_name} 
                ###final step
                graph_final[stop_id_time].append(dic)
            else:
                continue #for trip_id in graph[bus_name]:

5100 2_DROP OFFS ONLY 904846 ['5100_25320']
1759 2_NORTH TRANSFER 904846 ['1561_25223', '1877_25259', '1759_25281']
5100 2_DROP OFFS ONLY 904846 ['5100_25320']


In [ ]:
#graph_final only partial view for github upload page
#https://stackoverflow.com/questions/28704526/in-python-how-to-obtain-a-partial-view-of-a-dict
#[print(v) for i, v in enumerate(graph_final.items()) if i < 10]
{k: v for i, (k, v) in enumerate(graph_final.items()) if i < 10}

In [ ]:
#save_obj(graph_final, 'graph_final')


In [ ]:
#load_obj('graph_final')